In [1]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras 
from tensorflow.keras.layers import Dense, BatchNormalization, Dropout, Activation, LeakyReLU
from tensorflow.keras import layers
from tensorflow.keras.regularizers import l2
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler


In [2]:
df = pd.read_csv('train.csv')
tdf = pd.read_csv('test.csv')
print(df.shape, tdf.shape)


(891, 12) (418, 11)


In [3]:
dropped_columns = df.columns[df.isnull().sum() > 100]
df = df.drop(columns=dropped_columns)
tdf = tdf.drop(columns=dropped_columns)
print(df.shape, tdf.shape)

(891, 10) (418, 9)


In [4]:
df_numeric = df.select_dtypes(include=['number'])
df.loc[:, df_numeric.columns] = df_numeric.fillna(df_numeric.mean())

tdf_numeric = tdf.select_dtypes(include=['number'])
tdf.loc[:, tdf_numeric.columns] = tdf_numeric.fillna(tdf_numeric.mean())

df_categorical = df.select_dtypes(include=['object', 'category'])
df.loc[:, df_categorical.columns] = df_categorical.apply(lambda x: x.fillna(x.mode()[0]))

tdf_categorical = tdf.select_dtypes(include=['object', 'category'])
tdf.loc[:, tdf_categorical.columns] = tdf_categorical.apply(lambda x: x.fillna(x.mode()[0]))
print(df.shape, tdf.shape)

(891, 10) (418, 9)


In [5]:
df = df.drop(columns=['Name', 'Ticket'])
tdf = tdf.drop(columns=['Name', 'Ticket'])
print(df.shape, tdf.shape)


(891, 8) (418, 7)


In [6]:
df = pd.get_dummies(df)
tdf = pd.get_dummies(tdf)
print(df.shape, tdf.shape)


(891, 11) (418, 10)


In [7]:
X = df.drop(columns='Survived')
y = df['Survived']

In [8]:
scaler = StandardScaler()

X_train = scaler.fit_transform(X)
X_test = scaler.fit_transform(tdf)
print(X_train.shape, X_test.shape)

(891, 10) (418, 10)


In [17]:
from tensorflow.keras.optimizers import Adam, RMSprop, Nadam
model3 = Sequential()

model3.add(Dense(250, input_shape=(10,)))
model3.add(BatchNormalization())
model3.add(LeakyReLU(alpha=0.01))
model3.add(Dense(200))
model3.add(BatchNormalization())
model3.add(LeakyReLU(alpha=0.01))
model3.add(Dense(1, activation='sigmoid'))

optimizer = RMSprop(learning_rate=0.001)
model3.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
model3.summary()

c:\ML Algorithms\myenv\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
c:\ML Algorithms\myenv\Lib\site-packages\keras\src\layers\activations\leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_3 (Dense)                 │ (None, 250)            │         2,750 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 250)            │         1,000 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu (LeakyReLU)         │ (None, 250)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 200)            │        50,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 200)            │           800 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_1 (LeakyReLU)       │ (None, 200)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │           201 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 54,951 (214.65 KB)

 Trainable params: 54,051 (211.14 KB)

 Non-trainable params: 900 (3.52 KB)

In [18]:
history3 = model3.fit(X_train, y, epochs=150, validation_split=0.1, batch_size=64, verbose=1)

Epoch 1/150
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 0.6580 - loss: 0.6448 - val_accuracy: 0.8444 - val_loss: 0.5725
Epoch 2/150
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8058 - loss: 0.4306 - val_accuracy: 0.8000 - val_loss: 0.5549
Epoch 3/150
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8008 - loss: 0.4202 - val_accuracy: 0.8222 - val_loss: 0.5378
Epoch 4/150
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8191 - loss: 0.4212 - val_accuracy: 0.7889 - val_loss: 0.5079
Epoch 5/150
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7854 - loss: 0.4344 - val_accuracy: 0.8333 - val_loss: 0.4856
Epoch 6/150
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8113 - loss: 0.4134 - val_accuracy: 0.7889 - val_loss: 0.4788
Epoch 7/150
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8156 - loss: 0.3889 - val_accuracy: 0.8222 - val_loss: 0.4782
Epoch 8/150
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8211 - loss: 0.3914 - val_accuracy: 0.8333 - 

In [19]:
pred3 = model3.predict(X_test)

14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step


In [20]:
pred3 = np.where(pred3 > 0.5, 1, 0)
pred3 = pred3.flatten()
pred3.shape

(418,)

In [21]:
submission = pd.DataFrame({
    'PassengerId' : tdf['PassengerId'],
    'Survived' : pred3
})

submission.to_csv('submission.csv', index=False)